In [1]:
import pandas as pd
import numpy as np
import time
import quandl
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

quandl.ApiConfig.api_key = 'MdTYhiXQR_LnUHzq-qma'

def generate_x(ts, data, length, begin):
    norm = ts[data][begin]
    x = np.array([ts[data][begin:begin + length].tolist()]) / norm
    y = ts[data][begin + length] / norm
    return x, y


def generate_time_series(tinker, data, length, begin=0, x=None, y=None, debug_mode=False):
    comp = quandl.get(tinker)
    o = 0
    k = 0
    if x is None :
        x, y = generate_x(comp, data, length, begin)
        o = 1
    for i in range(begin + o, len(comp) - length - 1):
        norm = comp[data][i]
        x = np.concatenate((x, np.array([comp[data][i:i + length].tolist()]) / norm), axis=0)
        y = np.append(y, comp[data][i + length + 1] / norm)
        if debug_mode:
            k+=1
            if k % 1000 == 0:
                print(f'{tinker}: {k} datas')
    return x, y

def mean_percent_error(y_true, y_pred):
    return np.abs(100*(y_pred-y_true)/y_true)

In [2]:
tinkers = ['WIKI/AAPL', 'WIKI/AXP', 'WIKI/BA', 'WIKI/CAT', 'WIKI/CSCO', 'WIKI/CVX', 'WIKI/XOM',
           'WIKI/GS', 'WIKI/HD','WIKI/IBM','WIKI/INTC','WIKI/JNJ','WIKI/KO','WIKI/JPM','WIKI/MCD',
           'WIKI/MMM','WIKI/MRK','WIKI/NKE','WIKI/PG','WIKI/WMT','WIKI/V','WIKI/VZ','WIKI/DIS']

x, y = generate_time_series('WIKI/MSFT','Close', 24, 16)

for t in tinkers:
    x, y = generate_time_series(t, 'Close', 24, 16, x=x, y=y, debug_mode=True)

WIKI/AAPL: 1000 datas
WIKI/AAPL: 2000 datas
WIKI/AAPL: 3000 datas
WIKI/AAPL: 4000 datas
WIKI/AAPL: 5000 datas
WIKI/AAPL: 6000 datas
WIKI/AAPL: 7000 datas
WIKI/AAPL: 8000 datas
WIKI/AAPL: 9000 datas
WIKI/AXP: 1000 datas
WIKI/AXP: 2000 datas
WIKI/AXP: 3000 datas
WIKI/AXP: 4000 datas
WIKI/AXP: 5000 datas
WIKI/AXP: 6000 datas
WIKI/AXP: 7000 datas
WIKI/AXP: 8000 datas
WIKI/AXP: 9000 datas
WIKI/AXP: 10000 datas
WIKI/AXP: 11000 datas
WIKI/BA: 1000 datas
WIKI/BA: 2000 datas
WIKI/BA: 3000 datas
WIKI/BA: 4000 datas
WIKI/BA: 5000 datas
WIKI/BA: 6000 datas
WIKI/BA: 7000 datas
WIKI/BA: 8000 datas
WIKI/BA: 9000 datas
WIKI/BA: 10000 datas
WIKI/BA: 11000 datas
WIKI/BA: 12000 datas
WIKI/BA: 13000 datas
WIKI/BA: 14000 datas
WIKI/CAT: 1000 datas
WIKI/CAT: 2000 datas
WIKI/CAT: 3000 datas
WIKI/CAT: 4000 datas
WIKI/CAT: 5000 datas
WIKI/CAT: 6000 datas
WIKI/CAT: 7000 datas
WIKI/CAT: 8000 datas
WIKI/CAT: 9000 datas
WIKI/CAT: 10000 datas
WIKI/CAT: 11000 datas
WIKI/CAT: 12000 datas
WIKI/CAT: 13000 datas
WIKI/CA

In [3]:
x, y = shuffle(x,y)
print(x[10])
print(y[10])

[1.         0.99690323 0.97754839 0.96464516 0.96129032 0.96774194
 0.99354839 0.98709677 0.97419355 0.99354839 0.98709677 0.99045161
 0.984      0.97754839 0.99690323 0.99045161 1.00335484 1.01290323
 1.01625806 0.98709677 0.96464516 0.97109677 0.97419355 0.984     ]
0.9290322580645162


In [ ]:

NAME = f"MultiStockFeedfoward_32-96-32{int(time.time())}"

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(24,)))
model.add(layers.Dense(96, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

optimizer = optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer,
              loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit(x,y, batch_size=32, epochs=50, validation_split=0.2, metric=['percent-error', mean_percent_error],callbacks=[tensorboard])
